In [1]:
import pandas as pd
import numpy as np
import glob
import re

# lowz

sn2006bk: U band deleted, only 1 data point

sn2001cz: U band deleted, only 1 data point

sn1993l: U band deleted, only 1 data point

sn2003cq: U band deleted, only 1 data point

sn1999bm: I band deleted, only 1 data point

sn2000bh: U band deleted, only 1 data point

In [3]:
sne_directories = glob.glob('test_files/Sifto/SALT2/lowz/*')
sifto_params = pd.read_csv('test_files/Sifto/sifto.dat', delim_whitespace=True)

sn_count = 0
sn_exc_count = 0
for directory in sne_directories:
    sn_directory = glob.glob(f'{directory}/lc2fit*')
    name = directory.split('/')[-1]
    
    if name in sifto_params['#name'].values:     
        with open(f'{directory}/lightfile', 'r') as f:
            for line in f:
                if line.startswith('z_cmb'):
                    z = line.split()[-1]
                if line.startswith('RA'):
                    ra = line.split()[-1]
                if line.startswith('DEC'):
                    dec = line.split()[-1]

        mjd, flux, err, zps, bands = np.empty(0), np.empty(0), np.empty(0), [], []
        for band_file in sn_directory:
            band = re.split('lc2fit_|\.dat', band_file)[-2]
            if len(band)==1:
                band = 'Standard_'+band

            zp = 27
            d, m, e = np.loadtxt(band_file, skiprows=3).T
            f = 10**(-0.4*(m-zp))
            df = np.abs(f* (-0.4*np.log(10)*e))

            mjd, flux, err = np.r_[mjd, d], np.r_[flux, f], np.r_[err, df]
            zps.append([zp]*len(f))
            bands.append([band]*len(f))

        sn_df = pd.DataFrame(columns = ['mjd', 'flux' ,'flux_err', 'zp', 'band', 'mag_sys'])
        # create a flat list with bands and zps and mag_sys
        bands = [item for sublist in bands for item in sublist]
        zps = [item for sublist in zps for item in sublist]
        mag_sys = ['VEGA']*len(zps)
        sn_df.mjd, sn_df.flux, sn_df.flux_err, sn_df.zp, sn_df.band, sn_df.mag_sys = mjd, flux, err, zps, bands, mag_sys

        ##############
        save = True
        if save:
            data_file = 'data_sifto/'+name+'.dat'
            with open(data_file, 'w') as file:
                file.write('name z ra dec\n')
                file.write(f'{name} {z} {ra} {dec}\n')
            sn_df[['mjd', 'flux', 'flux_err', 'zp', 'band', 'mag_sys']].to_csv(data_file, index=False, sep=' ', mode='a')
        sn_count += 1

    else:
        #print(f'{name} not in Sifto!')
        sn_exc_count += 1
        pass
        
print(f'There are {sn_count} SNe with Sifto results')
print(f'There are {sn_exc_count} SNe without Sifto results')
print(f'Total: {sn_count + sn_exc_count} SNe')

There are 116 SNe with Sifto results
There are 168 SNe without Sifto results
Total: 284 SNe


## SDSS

In [4]:
sne_directories = glob.glob('test_files/Sifto/SALT2/SDSS/*')
sifto_params = pd.read_csv('test_files/Sifto/sifto.dat', delim_whitespace=True)

sn_count = 0
sn_exc_count = 0
sn_multi_count = 0
#sne_directories = [sne_directories[0]]
for directory in sne_directories:
    sn_directory = glob.glob(f'{directory}/lc2fit*')
    name = directory.split('/')[-1]
    
    if name in sifto_params['#name'].values:     
        with open(f'{directory}/lightfile', 'r') as f:
            for line in f:
                if line.startswith('z_cmb'):
                    z = line.split()[-1]
                if line.startswith('RA'):
                    ra = line.split()[-1]
                if line.startswith('DEC'):
                    dec = line.split()[-1]

        mjd, flux, err, zps, bands = np.empty(0), np.empty(0), np.empty(0), [], []
        for band_file in sn_directory:
            band = re.split('lc2fit_|\.dat', band_file)[-2]
            if len(band)==1:
                band = band+'_2.5m'
                
            d, f, df, zp = np.loadtxt(band_file, skiprows=3).T

            mjd, flux, err, zps = np.r_[mjd, d], np.r_[flux, f], np.r_[err, df], np.r_[zps, zp]
            bands.append([band]*len(f))

        sn_df = pd.DataFrame(columns = ['mjd', 'flux' ,'flux_err', 'zp', 'band', 'mag_sys'])
        # create a flat list with bands and mag_sys
        bands = [item for sublist in bands for item in sublist]
        mag_sys = ['AB']*len(bands)
        sn_df.mjd, sn_df.flux, sn_df.flux_err, sn_df.zp, sn_df.band, sn_df.mag_sys = mjd, flux, err, zps, bands, mag_sys

        ##############
        save = True
        if save and (name!='sn2005hc' and name!='sn2005hj' and name!='sn2005hk' and name!='sn2005ir'):
            data_file = 'data_sifto/'+name+'.dat'
            with open(data_file, 'w') as file:
                file.write('name z ra dec\n')
                file.write(f'{name} {z} {ra} {dec}\n')
            sn_df[['mjd', 'flux', 'flux_err', 'zp', 'band', 'mag_sys']].to_csv(data_file, index=False, sep=' ', mode='a')
            
        if name!='sn2005hc' and name!='sn2005hj' and name!='sn2005hk' and name!='sn2005ir':
            sn_count += 1
        else:
            sn_multi_count += 1

    elif not name.endswith('.txt'):
        #print(f'{name} not in Sifto!')
        sn_exc_count += 1
        pass
        
print(f'There are {sn_count} SNe with Sifto results ({sn_multi_count} with multiple data not counted)')
print(f'There are {sn_exc_count} SNe without Sifto results')
print(f'Total: {sn_count + sn_exc_count + sn_multi_count} SNe')

There are 91 SNe with Sifto results (2 with multiple data not counted)
There are 53 SNe without Sifto results
Total: 146 SNe


## SNLS

In [5]:
sne_directories = glob.glob('test_files/Sifto/SALT2/SNLS/*')
sifto_params = pd.read_csv('test_files/Sifto/sifto.dat', delim_whitespace=True)

sn_count = 0
sn_exc_count = 0
#sne_directories = [sne_directories[0]]
for directory in sne_directories:
    sn_directory = glob.glob(f'{directory}/lc2fit*')
    name = directory.split('/')[-1]
    
    if name in sifto_params['#name'].values:     
        with open(f'{directory}/lightfile', 'r') as f:
            for line in f:
                if line.startswith('z_cmb'):
                    z = line.split()[-1]
                if line.startswith('RA'):
                    ra = line.split()[-1]
                if line.startswith('DEC'):
                    dec = line.split()[-1]

        mjd, flux, err, zps, bands = np.empty(0), np.empty(0), np.empty(0), [], []
        for band_file in sn_directory:
            band = re.split('lc2fit_|\.dat', band_file)[-2]
            if len(band)==1:
                band = band+'sSAGEMrf4'
                
            d, f, df, zp = np.loadtxt(band_file, skiprows=13).T

            if not isinstance(f, float):
                mjd, flux, err, zps = np.r_[mjd, d], np.r_[flux, f], np.r_[err, df], np.r_[zps, zp]
                bands.append([band]*len(f))

        sn_df = pd.DataFrame(columns = ['mjd', 'flux' ,'flux_err', 'zp', 'band', 'mag_sys'])
        # create a flat list with bands and mag_sys
        bands = [item for sublist in bands for item in sublist]
        mag_sys = ['VEGA']*len(bands)
        sn_df.mjd, sn_df.flux, sn_df.flux_err, sn_df.zp, sn_df.band, sn_df.mag_sys = mjd, flux, err, zps, bands, mag_sys

        ##############
        save = False
        if save:
            data_file = 'data_sifto/'+name+'.dat'
            with open(data_file, 'w') as file:
                file.write('name z ra dec\n')
                file.write(f'{name} {z} {ra} {dec}\n')
            sn_df[['mjd', 'flux', 'flux_err', 'zp', 'band', 'mag_sys']].to_csv(data_file, index=False, sep=' ', mode='a')
        sn_count += 1

    elif (not name.endswith('.txt')) and (not name.startswith('README')):
        #print(f'{name} not in Sifto!')
        sn_exc_count += 1
        pass
        
print(f'There are {sn_count} SNe with Sifto results')
print(f'There are {sn_exc_count} SNe without Sifto results')
print(f'Total: {sn_count + sn_exc_count} SNe')

There are 241 SNe with Sifto results
There are 51 SNe without Sifto results
Total: 292 SNe


## HST

In [6]:
sne_directories = glob.glob('test_files/Sifto/SALT2/RiessHST/*')
sifto_params = pd.read_csv('test_files/Sifto/sifto.dat', delim_whitespace=True)

sn_count = 0
sn_exc_count = 0
#sne_directories = [sne_directories[0]]
for directory in sne_directories:
    sn_directory = glob.glob(f'{directory}/lc2fit*')
    name = directory.split('/')[-1]
    
    if name in sifto_params['#name'].values:     
        with open(f'{directory}/lightfile', 'r') as f:
            for line in f:
                if line.startswith('z_cmb'):
                    z = line.split()[-1]
                if line.startswith('RA'):
                    ra = line.split()[-1]
                if line.startswith('DEC'):
                    dec = line.split()[-1]

        mjd, flux, err, zps, bands = np.empty(0), np.empty(0), np.empty(0), [], []
        for band_file in sn_directory:
            band = re.split('lc2fit_|\.dat', band_file)[-2]
                
            d, f, df, zp = np.loadtxt(band_file, skiprows=3).T

            if not isinstance(f, float):
                mjd, flux, err, zps = np.r_[mjd, d], np.r_[flux, f], np.r_[err, df], np.r_[zps, zp]
                bands.append([band]*len(f))

        sn_df = pd.DataFrame(columns = ['mjd', 'flux' ,'flux_err', 'zp', 'band', 'mag_sys'])
        # create a flat list with bands and mag_sys
        bands = [item for sublist in bands for item in sublist]
        mag_sys = ['VEGA']*len(bands)
        sn_df.mjd, sn_df.flux, sn_df.flux_err, sn_df.zp, sn_df.band, sn_df.mag_sys = mjd, flux, err, zps, bands, mag_sys

        ##############
        save = False
        if save:
            data_file = 'data_sifto/'+name+'.dat'
            with open(data_file, 'w') as file:
                file.write('name z ra dec\n')
                file.write(f'{name} {z} {ra} {dec}\n')
            sn_df[['mjd', 'flux', 'flux_err', 'zp', 'band', 'mag_sys']].to_csv(data_file, index=False, sep=' ', mode='a')
        sn_count += 1

    else:
        #print(f'{name} not in Sifto!')
        sn_exc_count += 1
        pass
        
print(f'There are {sn_count} SNe with Sifto results')
print(f'There are {sn_exc_count} SNe without Sifto results')
print(f'Total: {sn_count + sn_exc_count} SNe')

There are 14 SNe with Sifto results
There are 26 SNe without Sifto results
Total: 40 SNe


There is a total of 766 SNe and 462 with Sifto results.